all the previous algs rely mainly on data 

but in hidden_markov_model we deal with probability distributions

the best example is a basic weather model that depends on the previous weather cases to predict the future weather

Hidden Markov Models

"The Hidden Markov Model is a finite set of states, each of which is associated with a (generally multidimensional) probability distribution.

Transitions among the states are governed by a set of probabilities called transition probabilities:
(http://jedlik.phy.bme.hu/gerjanos/HMM/node4.html)

A hidden markov model works with probabilities to predict future events or states. In this section we will learn how to create a hidden markov
model that can predict the weather.

This section is based on the following TensorFlow tutorial.
https://www.tensortflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel

so the states in the weather model are hot day and cold day (these are what we call hidden)-->bec we never interact with those states while interacting with the model

# Data

Let's start by discussing the type of data we use when we work with a hidden markov model.

In the previous sections we worked with large datasets of 100's of different entries. For a markov model we are only interested in probability
distributions that have to do with states.

We can find these probabilities from large datasets or may already have these values. We'll run through an example in a second that should
clear some things up, but let's discuss   ~the components of a markov model~.


~States~: In each markov model we have a finite set of states. These states could be something like "warm' and 'cold" or "high" and "low" or even
'red', 'green' and "blue'. These states are "hidden' within the model, which means we do not directly observe them.

~Observations~: Each state has a particular outcome or observation associated with it based on a probability distribution. An example of this is
the following: On a hot day Tim has a 80% chance of being happy and a 20% chance of being sad.

~Transitions~: Each state will have a probability defining the likelyhood of transitioning to a different state. An example is the following: a cold day
has a 30% chance of being followed by a hot day and a 70% chance of being followed by another cold day.

To create a hidden markov model we need.
.States
.Observation Distribution
.Transition Distribution

For our purpose we will assume we already have this information available as we attempt to predict the weather on a given day.

In [9]:
import tensorflow as tf
import tensorflow_probability as tfp 


[**Weather Model**]




Taken directly from the TensorFlow documentation
(https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel).

We will model a simple weather system and try to predict the temperature on each day given the following information.

1. Cold days are encoded by a 0 and hot days are encoded by a 1.
2. The first day in our sequence has an 80% chance of being cold.
3. A cold day has a 30% chance of being followed by a hot day.
4. A hot day has a 20% chance of being followed by a cold day.
5. On each day the temperature is normally distributed with mean and standard deviation 0 and 5 ona cold day and mean and standard deviation 15 and 10 on a
t day.

if you're unfamiliar with standard deviation it can be put simply as the range of expected values.

In this example, on a hot day the average temperature is 15 and ranges from 5 to 25.

To model this in TensorFlow we will do the following.

In [10]:
tfd= tfp.distributions       # making a shortcut for later on
initial_distribution = tfd.Categorical(probs=[0.8,0.2])  # Refer to point 2 above
    ##so that the initial distribution of being cold is 80% and 20% after
transition_distribution = tfd.Categorical (probs=[[0.7, 0.3],
                                                  [0.2, 0.8]]) # refer to points 3 and 4 above
observation_distribution= tfd.Normal(loc=[0.,15.], scale=[5.,10.])   # refer to point 5 above
    # the loc argument represents the mean(the mean is 0 on cold days and 15 on hot) and the Scale is the standard devitation(the std deviation on hot days is 10 and 5 on cold )

In [11]:
#We've now created distribution variables to model our system and it's time to create the hidden markov model.
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution= observation_distribution,
    num_steps=7)        ##steps is how many days we want to predict for  

In [12]:
#The number of steps represents the number of days that we would like to predict intormation for. In this case we ve chosen 7, an entire week.
#To get the expected temperatures on each day we can do the following.


mean = model.mean()
# due to the way TensorFlow Works on a lower level we need to evaluate part of the graph
# from within a session to see the value of this tensor
#in the new version of tensorflow we need to use tf. compat . v1. Session (0 rather than just tf. Session ()
with tf.compat.v1.Session() as sess:
    print(mean.numpy())


[3.        5.9999995 7.4999995 8.25      8.625     8.812501  8.90625  ]


هنا عمر مالقيم هتتغير عشان الاحتمالات ثابتة 


.او حرارة 3 عشان انتا بادئ بيوم سقعة


ولو غيرت الاحتمالات اللي فوق في الinitial distribution
هتغير القيم 

